In [9]:
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from src.face_recognition.pipelines.data_science.face_recognition import preprocessing, FaceFeaturesExtractor, FaceRecogniser, evaluate
import tqdm
import joblib
from torchvision import transforms, datasets
from PIL import Image


In [10]:
def dataset_to_embeddings(dataset, features_extractor):
    transform = transforms.Compose([
        preprocessing.ExifOrientationNormalize(),
        transforms.Resize(1024)
    ])

    embeddings = []
    labels = []
    for img_path, label in tqdm.tqdm(dataset.samples):
        _, embedding = features_extractor(transform(Image.open(img_path).convert('RGB')))
        if embedding is None:
            continue
        if embedding.shape[0] > 1:
            embedding = embedding[0, :]
        embeddings.append(embedding.flatten())
        labels.append(label)

    return np.stack(embeddings), labels



def normalise_string(string):
    return string.lower().replace(' ', '_')

def normalise_dict_keys(dictionary):
    new_dict = dict()
    for key in dictionary.keys():
        new_dict[normalise_string(key)] = dictionary[key]
    return new_dict


def prepare_embeddings(train_path: str, augmented_train_path: str, augment: bool):
    print("generating embeddings...")
    features_extractor = FaceFeaturesExtractor()

    # use augmented dataset if augment is True
    data_path = augmented_train_path if augment else train_path
    dataset = datasets.ImageFolder(data_path)

    # extract embeddings
    embeddings, labels = dataset_to_embeddings(dataset, features_extractor)
    print(embeddings.shape, len(labels))
    # normalise labels
    dataset.class_to_idx = normalise_dict_keys(dataset.class_to_idx)
    idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
    labels = list(map(lambda idx: idx_to_class[idx], labels))
    
    return embeddings, np.array(labels, dtype=np.str).reshape(-1, 1), dataset.class_to_idx

In [11]:
# import libraries i need for train function below

def train(embeddings, labels, class_to_idx, grid_search:bool):
    print("training model...")
    features_extractor = FaceFeaturesExtractor()
    
    softmax = LogisticRegression(solver='lbfgs', multi_class='multinomial', C=10, max_iter=10000, verbose=1)
    if grid_search:
        clf = GridSearchCV(
            estimator=softmax,
            param_grid={'C': [0.1, 1, 10, 100, 1000]},

            cv=3
        )
    else:
        clf = softmax
    clf.fit(embeddings, labels)
    if grid_search:
        print("Best parameters set found on training set:")
        print(clf.best_params_)

    labels = labels.tolist()
    idx_to_class = {v: k for k, v in class_to_idx.items()}

    target_names = list(map(lambda i: i[1], sorted(idx_to_class.items(), key=lambda i: i[0])))
    report = metrics.classification_report(labels, clf.predict(embeddings), target_names=target_names, output_dict=True)
    print("Training report:")
    print(f"Accuracy: {report['accuracy']:.3f}")
    print(f"Precision: {report['weighted avg']['precision']:.3f}")
    print(f"Recall: {report['weighted avg']['recall']:.3f}")
    print(f"F1-score: {report['weighted avg']['f1-score']:.3f}")

    return FaceRecogniser(features_extractor, clf, idx_to_class)

In [12]:
train_path = os.path.join('..', 'data/01_raw/data_dev')
test_path = os.path.join('..', 'data/01_raw/test_dev')

In [13]:
# generate embeddings for train and test datasets

train_embeddings, train_labels, train_class_to_idx = prepare_embeddings(train_path, train_path, False)
# train model
recogniser = train(train_embeddings, train_labels, train_class_to_idx, False)


generating embeddings...


  0%|          | 0/6 [00:00<?, ?it/s]c:\Users\bilal\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
100%|██████████| 6/6 [00:03<00:00,  1.71it/s]
C:\Users\bilal\AppData\Local\Temp\ipykernel_8896\2335227457.py:48: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return embeddings, np.array(labels, dtype=np.str).reshape(-1, 1), dataset.class_to_idx


(6, 512) 6
training model...
Training report:
Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1-score: 1.000


c:\Users\bilal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [14]:
test_embeddings, test_labels, test_class_to_idx = prepare_embeddings(test_path, test_path, False)

generating embeddings...


  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\bilal\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
100%|██████████| 3/3 [00:01<00:00,  1.61it/s]

(3, 512) 3



C:\Users\bilal\AppData\Local\Temp\ipykernel_8896\2335227457.py:48: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return embeddings, np.array(labels, dtype=np.str).reshape(-1, 1), dataset.class_to_idx


In [15]:
def eval(model:FaceRecogniser, embeddings, labels, class_to_idx, grid_search:bool, augment:bool):
    metrics_dict = {}

    labels = labels.tolist()

    idx_to_class = {v: k for k, v in class_to_idx.items()}
    target_names =list(map(lambda i: i[1], sorted(idx_to_class.items(), key=lambda i: i[0])))
    print(target_names)
    # save the metrics to a csv file
    gs=1 if grid_search else 0
    ag=1 if augment else 0
    # define metrics
    print(labels)
    metrics = evaluate.ModelMetrics(model)
    report, avg_proba = metrics.calculate_metrics(embeddings, labels, target_names)
    print("Test report:")
    print(f"Accuracy: {report['accuracy']:.3f}")
    print(f"Precision: {report['weighted avg']['precision']:.3f}")
    print(f"Recall: {report['weighted avg']['recall']:.3f}")
    print(f"F1-score: {report['weighted avg']['f1-score']:.3f}")
    print(f"Average confidence: {avg_proba * 100}%")
    
    metrics_dict = {
        "Accuracy": f"{report['accuracy']:.3f}",
        "Precision": f"{report['weighted avg']['precision']:.3f}",
        "Recall": f"{report['weighted avg']['recall']:.3f}",
        "F1-score": f"{report['weighted avg']['f1-score']:.3f}",
        "Average confidence": f"{avg_proba * 100}%",
        'grid_search': gs,
        'augmentation': ag
    }
    
    
    # convert metrics to dataframe
    metrics_df = pd.DataFrame(metrics_dict, index=[0])
    return metrics_df

In [16]:
# evaluate model
metrics_df = eval(recogniser, test_embeddings, test_labels, test_class_to_idx, False, False)
metrics_df


['alisha_richman', 'alison_krauss', 'alison_lohman']
[['alisha_richman'], ['alison_krauss'], ['alison_lohman']]
['alisha_richman' 'alison_krauss' 'alison_lohman']
[['alisha_richman'], ['alison_krauss'], ['alison_lohman']]
Test report:
Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1-score: 1.000
Average confidence: 16.666666666666664%


,Accuracy,Precision,Recall,F1-score,Average confidence,grid_search,augmentation
0,1.000,1.000,1.000,1.000,16.666666666666664%,0,0
